In [1]:
import sys
import numpy as np
# import scikitlearn
import matplotlib
import pandas as pd
import glob

# Local includes:
sys.path.append("../../src")
import therm_params as tp
from power_model import (
    leakage_power,
    peripheral_power,
    board_power
)


/usr/local/lib/python3.5/dist-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [2]:
blackscholes_files = glob.glob("/home/mark/Documents/Repos/userland_dvfs_gov/data/blackscholes_benchmarking/*.csv")
bodytrack_files = glob.glob("/home/mark/Documents/Repos/userland_dvfs_gov/data/bodytrack_benchmarking/*.csv")
files = blackscholes_files

In [14]:
df_list = []
for file in files:
    df = pd.read_csv(file, index_col=False, header=0, sep='\t')
    df_list.append(df)

# for file in bodytrack_files:
#     df = pd.read_csv(file, index_col=False, header=0, sep='\t')
#     df_list.append(df)

data = pd.concat(df_list, ignore_index=True)
# Drop all measurements where watts <= 0:
data = data.loc[data.watts > 0]
# fill in the potentially missing voltage values
data.volts_big_cluster = data.freq_big_cluster.map(lambda x: tp.big_f_to_v[float(x)/1000000000])
data.volts_little_cluster = data.freq_little_cluster.map(lambda x: tp.little_f_to_v[float(x)/1000000000])
# Replace with correct value for gpu and mem voltage:
data.volts_gpu = 1.0
data.volts_mem = 1.2
# Recompute the leakage power:
data.['w_leak2'] = data.apply(lambda row: leakage_power(tp.c1, tp.c2, tp.igate, row.), axis=1)

# data.drop(["volts_big_cluster", "volts_little_cluster", "volts_mem", "volts_gpu"], axis=1, inplace=True)

data_filtered = pd.DataFrame()
data_filtered["usage_little"] = data[["usage_c0", "usage_c1", "usage_c2", "usage_c3"]].apply(func = np.sum, axis=1)
data_filtered["usage_big"] = data[["usage_c4", "usage_c5", "usage_c6", "usage_c7"]].apply(func = np.sum, axis=1)
data_filtered["max_temp_big"] = data[["temp4", "temp5", "temp6", "temp7"]].apply(func = np.max)
data_filtered["avg_temp_big"] = data[["temp4", "temp5", "temp6", "temp7"]].apply(func = np.mean)
data_filtered["gpu_temp"] = data.temp_gpu
data_filtered["vvf_b"] = data.freq_big_cluster.map(lambda x: tp.big_f_to_v[float(x)/1000000000]**2 * x)
data_filtered["vvf_l"] = data.freq_little_cluster.map(lambda x: tp.little_f_to_v[float(x)/1000000000]**2 * x)

In [15]:
data.columns

Index(['freq_big_cluster', 'freq_gpu', 'freq_little_cluster', 'freq_mem',
       'temp4', 'temp5', 'temp6', 'temp7', 'temp_gpu', 'time', 'usage_c0',
       'usage_c1', 'usage_c2', 'usage_c3', 'usage_c4', 'usage_c5', 'usage_c6',
       'usage_c7', 'volts_big_cluster', 'volts_gpu', 'volts_little_cluster',
       'volts_mem', 'w_dyn', 'w_leak', 'w_periph', 'watts'],
      dtype='object')

In [ ]:
tp.little_f_to_v